<ul>
    <li> <h2><b>Universidade de Brasília - UnB</b></h2> </li>
    <li> <b>Campus UnB Gama</b> </li>
    <li> <b>Disciplina: Processamento Digital de Sinais Financeiros</b> </li>
    <a href="https://www.linkedin.com/in/marcelino-andrade-b164b369/">
    <li> <b>Prof. Marcelino Monteiro de Andrade Dr.</b> </li>
</ul>


    Segunda (1/5) Atividade Prática Avaliativa
    Entrega da Atividade 10/11/2019 
    "A não entrega na data esperada reduz em 50% na nota da avaliação"


<div id="Tópicos">
    <h2>Questões</h2>
</div>&ensp;
    <ol> 
        <li>Implemente uma estratégia de trade automático baseado em duas médias móveis. Utilize um período mínimo de 3 anos, otimize nos dois primeiros anos e aplique no terceiro. Utilize os ativos escolhidos na primeira avaliação, considere um capital de R$100.000,00 e as taxas envolvidas na operação.</a></li>&ensp;
        <li>Implemente uma estratégia de trade automático baseado em duas médias móveis anteriores, adicionando à aplicação o RSI ou MACD. Utilize um período mínimo de 3 anos, otimize nos dois primeiros anos e aplique no terceiro. Utilize os ativos escolhidos na questão 1.</a></li>&ensp;
        <li>Escolha uma estratégia de trade automático de contra tendência, apresentada na Referência Palex. Utilize um período mínimo de 3 anos, otimize nos dois primeiros anos e aplique no terceiro. Utilize os ativos escolhidos na questão 1.</a></li>&ensp;
        <li>Compare os resultados das três estrategias de trades, quanto: a) Retorno de Capital, b) Número de Trades, c) Taxa de Acertos, d) Fator de Lucro, e) Índice Sharpe e o f) retorno dos ativos avaliados na condição buy&hold.</a></li>&ensp;
        <li>Observação: caso a equipe tenha o interesse em avaliar outras estratégias de algo trading e bem vindo também!</a></li>&ensp;        
</ol>



In [ ]:
%matplotlib inline 
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from tqdm import tqdm
import investpy
import matplotlib.dates as mdates
from scipy import signal
import random
import math

import warnings as wrn
wrn.filterwarnings('ignore', category = DeprecationWarning) 
wrn.filterwarnings('ignore', category = FutureWarning) 
wrn.filterwarnings('ignore', category = UserWarning)


def sumper(per):
    persum=100*(np.exp(np.log1p(per/100).cumsum())-1)
    return persum

In [ ]:
stocks=[]

out_otim = None

for stk in tqdm(stocks):

    df_otim = investpy.get_stock_historical_data(stock=stk,
                                            country='Brazil',
                                            from_date='16/11/2017',
                                            to_date='16/11/2019') #16/11/2019

    if out_otim is None:
        out_otim=df_otim["Close"]
    else:
        out_otim=pd.concat([out_otim,df_otim["Close"]],axis=1)
out_otim.columns=stocks
out_otim.tail()

In [ ]:
import missingno as msno
msno.matrix(df_otim)

# Peso de cada ativo na carteira

In [ ]:
CapitalInicial=[]

CapitalInicial.append(0.51748124) #IRBR3

CapitalInicial.append(0.23356792) #RENT3

CapitalInicial.append(0.07458088) #MGLU3

CapitalInicial.append(0.17436996) #JBSS3


# QUESTÃO 1

## Atribuir valores aleatórios para as médias short e long para otimizar e escolher as métricas que possuem maior rendimento 

# Operar comprado e vendido

In [ ]:
QtdTrades=[]
Retornos=[]
retornos_aux=[]
melhor_short=0
melhor_long=0
melhor_retorno=0
intervalo_short=25-1+1
intervalo_long=85-26+1
valores_short=[]
valores_long=[]
for iteracao in tqdm(range(1,(intervalo_short*intervalo_long+1))):
    
    if (iteracao/intervalo_long) == int(iteracao/intervalo_long):
        short=int(iteracao/intervalo_long)
    else:
        short=int(iteracao/intervalo_long)+1
    
    long=iteracao-(intervalo_long*(short-1))+intervalo_short
    
    soma_retornos=0
    soma_retornosComprado=0
    soma_retornosVendido=0
    Quantidade_trades=0
    i2=0
    for stk in stocks:
        trades = pd.DataFrame(index=out_otim.index)
        trades['Close']=out_otim[stk].values


        trades['short'] = trades['Close'].rolling(window=short).mean()
        trades['long'] = trades['Close'].rolling(window=long).mean()
        
        
        trades['signalBuy']=0
        trades['signalSell']=0

 #       buy=0
 #       buyed=0
 #       sold=0
 #
 #       for day in range(1,len(trades)):
 #           if sold==False:
 #               if (trades['short'][day]>trades['long'][day]):
 #                   buy = 1  
 #               if buy == True:
 #                   if (trades['short'][day]<trades['long'][day]):
 #                       buy = 0
 #                   trades['signalBuy'][day]=buy

 #           if buy==False & trades['signalBuy'][day-1]==0:
 #               if (trades['short'][day]<trades['long'][day]):
 #                   sold = 1      
 #               if sold == True:
 #                   if (trades['short'][day]>trades['long'][day]):
 #                       sold = 0
 #                   trades['signalSell'][day]=sold

        #se estiver comprado no último dia vende
    
        buy=0
        buyed=0
        sold=0
        hold=False
        diasParado=0
        for day in range(1,len(trades)):
            if day>(diasParado+5):

                if sold==False & hold==False:
                    if (trades['short'][day]>trades['long'][day]):
                        buy = 1  
                    if buy == True:
                        if (trades['short'][day]<trades['long'][day]):
                            buy = 0
                            hold=True
                            diasParado=day
                        trades['signalBuy'][day]=buy

                if buy==False & hold==False:
                    if (trades['short'][day]<trades['long'][day]):
                        sold = 1      
                    if sold == True:
                        if (trades['short'][day]>trades['long'][day]):
                            sold = 0
                            hold=True
                            diasParado=day
                        trades['signalSell'][day]=sold
                hold=False

        if trades['signalBuy'][-1]==1 or trades['signalSell'][-1]==1:
            trades['signalBuy'][-1]=0
            trades['signalSell'][-1]=0

        # Posição dos trades
        trades['positionsBuy'] = trades['signalBuy'].diff()
        trades['positionsSell'] = trades['signalSell'].diff()
    
        pos_buy_buyed=np.where(trades['positionsBuy']==1)
        pos_sell_buyed=trades['positionsBuy']==-1
        days_buy_buyed=trades['Close'][trades['positionsBuy']==1].index
        days_sell_buyed=trades['Close'][trades['positionsBuy']==-1].index
        distance_buyed=days_sell_buyed-days_buy_buyed
        #
        pos_buy_sold=np.where(trades['positionsSell']==1)
        pos_sell_sold=trades['positionsSell']==-1
        days_buy_sold=trades['Close'][trades['positionsSell']==1].index
        days_sell_sold=trades['Close'][trades['positionsSell']==-1].index
        distance_sold=days_sell_sold-days_buy_sold
        #
        buy_trade=[]
        sell_trade=[]
        
        # Gráfico para comprado
        for i in range(len(days_buy_buyed)):
            values_2=trades['Close'][trades['Close'].index==(days_buy_buyed[i]+distance_buyed[i])].values[0]
            values_1=trades['Close'][trades['Close'].index==days_buy_buyed[i]].values[0]
            buy_trade.append(100*(values_2-values_1)/values_1)

        valuesbuy = np.array(buy_trade)
        values1buy=buy_trade
        
        #Gráfico para vendido
        for i in range(len(days_buy_sold)):
            values_2=trades['Close'][trades['Close'].index==(days_buy_sold[i]+distance_sold[i])].values[0]
            values_1=trades['Close'][trades['Close'].index==days_buy_sold[i]].values[0]
            sell_trade.append(100*(values_1-values_2)/values_1)

        valuessell = np.array(sell_trade)
        values1sell=sell_trade     
            
        a=1
        for kint in range(len(valuesbuy)):
            a*=((valuesbuy[kint]/100)+1)
        soma_retornosComprado+=(a*CapitalInicial[i2])

        b=1
        for kint in range(len(valuessell)):
            b*=((valuessell[kint]/100)+1)
        soma_retornosVendido+=(b*CapitalInicial[i2])

        soma_retornos+=((soma_retornosComprado*soma_retornosVendido)-1)

        i2+=1
        
        #Quantidade_trades=len(valuesbuy) + len(valuessell)
        
        
    #if iteracao==0:
     #   Retornos.append(-1000)
    
    Retornos.append(soma_retornos*100)
    valores_short.append(short)
    valores_long.append(long)
    retornos_aux=sorted(Retornos,reverse=True)
    
    #QtdTrades.append(Quantidade_trades)
    if retornos_aux[0]==(soma_retornos*100):
        melhor_short=short
        melhor_long=long
        melhor_retorno=soma_retornos*100
    
    
print(melhor_short,melhor_long,melhor_retorno)

# Gráfico com short entre 1 e 15; long entre 16 e 60

In [ ]:
fig = plt.subplots(figsize=(40,20))
plt.style.use('seaborn-whitegrid')

ax1 = plt.subplot(311)
ax1.plot(valores_short, label='Short')
ax1.plot(valores_long, label='Long')
ax1.plot(Retornos, label='Retorno')
ax1.set_ylabel("Retorno [%]",fontsize=25)
ax1.set_xlabel("Número de testes",fontsize=25)
titulo="Maior retorno " + str(round(melhor_retorno,2)) + "%, " + "Short: " + str(melhor_short) + " Long: " + str(melhor_long)
ax1.set_title(titulo,fontsize=25)

ax2 = plt.subplot(312)
sns.barplot(x=valores_short, y=Retornos) 
ax2.set_ylabel("Retorno [%]",fontsize=25)
ax2.set_xlabel("Valor short",fontsize=25)

ax3 = plt.subplot(313)
sns.barplot(x=valores_long, y=Retornos) 
ax3.set_ylabel("Retorno [%]",fontsize=25)
ax3.set_xlabel("Valor long",fontsize=25)

# Gráfico com short entre 1 e 15; long entre 16 e 85

In [ ]:
fig = plt.subplots(figsize=(40,20))
plt.style.use('seaborn-whitegrid')

ax1 = plt.subplot(311)
ax1.plot(valores_short, label='Short')
ax1.plot(valores_long, label='Long')
ax1.plot(Retornos, label='Retorno')
ax1.set_ylabel("Retorno [%]",fontsize=25)
ax1.set_xlabel("Número de testes",fontsize=25)
titulo="Maior retorno " + str(round(melhor_retorno,2)) + "%, " + "Short: " + str(melhor_short) + " Long: " + str(melhor_long)
ax1.set_title(titulo,fontsize=25)

ax2 = plt.subplot(312)
sns.barplot(x=valores_short, y=Retornos) 
ax2.set_ylabel("Retorno [%]",fontsize=25)
ax2.set_xlabel("Valor short",fontsize=25)

ax3 = plt.subplot(313)
sns.barplot(x=valores_long, y=Retornos) 
ax3.set_ylabel("Retorno [%]",fontsize=25)
ax3.set_xlabel("Valor long",fontsize=25)

# Gráfico com short entre 1 e 25; long entre 26 e 85

In [ ]:
fig = plt.subplots(figsize=(40,20))
plt.style.use('seaborn-whitegrid')

ax1 = plt.subplot(311)
ax1.plot(valores_short, label='Short')
ax1.plot(valores_long, label='Long')
ax1.plot(Retornos, label='Retorno')
ax1.set_ylabel("Retorno [%]",fontsize=25)
ax1.set_xlabel("Número de testes",fontsize=25)
titulo="Maior retorno " + str(round(melhor_retorno,2)) + "%, " + "Short: " + str(melhor_short) + " Long: " + str(melhor_long)
ax1.set_title(titulo,fontsize=25)

ax2 = plt.subplot(312)
sns.barplot(x=valores_short, y=Retornos) 
ax2.set_ylabel("Retorno [%]",fontsize=25)
ax2.set_xlabel("Valor short",fontsize=25)

ax3 = plt.subplot(313)
sns.barplot(x=valores_long, y=Retornos) 
ax3.set_ylabel("Retorno [%]",fontsize=25)
ax3.set_xlabel("Valor long",fontsize=25)

# Gráficos com as métricas obtidas na otimização

## Operar comprado e vendido

In [ ]:
i12=0
soma_retornosA=0
soma_retornosB=0
aux=[]

taxa_acerto=0
num_trades=0

fator_lucro=1
ftr_pos=1
ftr_neg=1

for stk in stocks:
    trades = pd.DataFrame(index=out_otim.index)
    trades['Close']=out_otim[stk].values

    short=melhor_short
    long=melhor_long

    trades['short'] = trades['Close'].rolling(window=short).mean()
    trades['long'] = trades['Close'].rolling(window=long).mean()
    
    trades['signalBuy']=0
    trades['signalSell']=0

    buy=0
    buyed=0
    sold=0
    hold=False
    diasParado=0
    for day in range(1,len(trades)):
        if day>(diasParado+5):
            
            if sold==False & hold==False:
                if (trades['short'][day]>trades['long'][day]):
                    buy = 1  
                if buy == True:
                    if (trades['short'][day]<trades['long'][day]):
                        buy = 0
                        hold=True
                        diasParado=day
                    trades['signalBuy'][day]=buy

            if buy==False & hold==False:
                if (trades['short'][day]<trades['long'][day]):
                    sold = 1      
                if sold == True:
                    if (trades['short'][day]>trades['long'][day]):
                        sold = 0
                        hold=True
                        diasParado=day
                    trades['signalSell'][day]=sold
            hold=False
                
    #se estiver comprado no último dia vende

    if trades['signalBuy'][-1]==1 or trades['signalSell'][-1]==1:
        trades['signalBuy'][-1]=0
        trades['signalSell'][-1]=0

    # Posição dos trades
    trades['positionsBuy'] = trades['signalBuy'].diff()
    trades['positionsSell'] = trades['signalSell'].diff()
    
    fig = plt.subplots(figsize=(30,20))
    plt.style.use('seaborn-whitegrid')

    ax1 = plt.subplot(321)
    ax1.plot(trades['Close'], label='Close')
    ax1.plot(trades['short'], label='Short')
    ax1.plot(trades['long'],label='Long')

    ax1.plot(trades.loc[trades.positionsBuy == 1.0].index, 
             trades.short[trades.positionsBuy == 1.0],
             '^', markersize=10, color='m',label='Buy')
    ax1.plot(trades.loc[trades.positionsBuy == -1.0].index, 
             trades.short[trades.positionsBuy == -1.0],
             'v', markersize=10, color='k',label='Sell') 

    ax1.plot(trades.loc[trades.positionsSell == 1.0].index, 
             trades.short[trades.positionsSell == 1.0],
             '^', markersize=10, color='r',label='Sell')
    ax1.plot(trades.loc[trades.positionsSell == -1.0].index, 
             trades.short[trades.positionsSell == -1.0],
             'v', markersize=10, color='g',label='Buy') 


    ax1.set_ylabel("Preço [R$]",fontsize=25)
    ax1.set_xlabel("Data",fontsize=25)
    ax1.set_title("Médias" + " " + stk,fontsize=25)
    ax1.legend(loc='upper left',fontsize=25)

    ax5 = plt.subplot(323)
    #
    pos_buy_buyed=np.where(trades['positionsBuy']==1)
    pos_sell_buyed=trades['positionsBuy']==-1
    days_buy_buyed=trades['Close'][trades['positionsBuy']==1].index
    days_sell_buyed=trades['Close'][trades['positionsBuy']==-1].index
    distance_buyed=days_sell_buyed-days_buy_buyed
    #
    pos_buy_sold=np.where(trades['positionsSell']==1)
    pos_sell_sold=trades['positionsSell']==-1
    days_buy_sold=trades['Close'][trades['positionsSell']==1].index
    days_sell_sold=trades['Close'][trades['positionsSell']==-1].index
    distance_sold=days_sell_sold-days_buy_sold
    #
    buy_trade=[]
    sell_trade=[]
    # Gráfico para comprado
    for i in range(len(days_buy_buyed)):
        values_2=trades['Close'][trades['Close'].index==(days_buy_buyed[i]+distance_buyed[i])].values[0]
        values_1=trades['Close'][trades['Close'].index==days_buy_buyed[i]].values[0]
        buy_trade.append(100*(values_2-values_1)/values_1)

    valuesbuy = np.array(buy_trade)
    values1buy=buy_trade

    idx = np.array(list(range(len(buy_trade)))) 
    clrs = ['red' if (x < 0) else 'blue' for x in valuesbuy]
    sns.barplot(x=idx, y=valuesbuy, palette=clrs) 

    plt.ylabel('Retorno R[%] Comprado',fontsize=25)
    plt.xlabel('Trades Comprado',fontsize=25)

    ax52 = ax5.twinx()

    ax52.plot(sumper(valuesbuy),'gold',linewidth=4)

    plt.ylabel('Retorno Acumulado R[$]',fontsize=25)
    plt.xlabel('Trades',fontsize=25)
    plt.title('Retorno ' + " " + stk,fontsize=25)

    #Gráfico para vendido
    ax5segundo = plt.subplot(325)
    
    for i in range(len(days_buy_sold)):
        values_2=trades['Close'][trades['Close'].index==(days_buy_sold[i]+distance_sold[i])].values[0]
        values_1=trades['Close'][trades['Close'].index==days_buy_sold[i]].values[0]
        sell_trade.append(100*(values_1-values_2)/values_1)

    valuessell = np.array(sell_trade)
    values1sell=sell_trade

    idx = np.array(list(range(len(sell_trade)))) 
    clrs = ['red' if (x < 0) else 'blue' for x in valuessell]
    sns.barplot(x=idx, y=valuessell, palette=clrs) 

    plt.ylabel('Retorno R[%] Vendido',fontsize=25)
    plt.xlabel('Trades Vendido',fontsize=25)

    ax5segundo2 = ax5segundo.twinx()

    ax5segundo2.plot(sumper(valuessell),'gold',linewidth=4)

    plt.ylabel('Retorno Acumulado R[$]',fontsize=25)
    plt.xlabel('Trades',fontsize=25)
    plt.title('Retorno ' + " " + stk,fontsize=25)
    
    plt.tight_layout()
    
    a=1
    for kint in range(len(valuesbuy)):
        a*=((valuesbuy[kint]/100)+1)
        if valuesbuy[kint]>0:
            taxa_acerto+=1
            ftr_pos*=((valuesbuy[kint]/100)+1)
        else:
            ftr_neg*=((valuesbuy[kint]/-100)+1)
    soma_retornosA+=(a*CapitalInicial[i12])
    
    b=1
    for kint in range(len(valuessell)):
        b*=((valuessell[kint]/100)+1)
        if valuessell[kint]>0:
            taxa_acerto+=1
            ftr_pos*=((valuessell[kint]/100)+1)
        else:
            ftr_neg*=((valuessell[kint]/-100)+1)
    soma_retornosB+=(b*CapitalInicial[i12])
    
    aux.append(((a*b)-1)*100)
  
    i12+=1
    
    num_trades+=(len(valuesbuy)+len(valuessell))

for iteracao in range(len(aux)):
    print(aux[iteracao], stocks[iteracao])
    
taxa_acerto=(taxa_acerto/num_trades)*100

fator_lucro= ((ftr_pos-1)*100000)/((ftr_neg-1)*100000)

In [ ]:
print(round(fator_lucro,2))

In [ ]:
print("Taxa de acerto: {0}%" .format(round(taxa_acerto,2)))

# Aplicando no período de teste

In [ ]:
stocks=['IRBR3','RENT3','MGLU3', 'JBSS3']

out_teste = None

for stk in tqdm(stocks):

    df_teste = investpy.get_stock_historical_data(stock=stk,
                                            country='Brazil',
                                            from_date='16/11/2019',
                                            to_date='16/11/2020')

    if out_teste is None:
        out_teste=df_teste["Close"]
    else:
        out_teste=pd.concat([out_teste,df_teste["Close"]],axis=1)
out_teste.columns=stocks
out_teste.tail()

In [ ]:
import missingno as msno
msno.matrix(df_teste)

## Operar comprado e vendido

In [ ]:
Trades=[]
Trades.append(0)
Trades.append(0)
Trades.append(0)
Trades.append(0)
i12=0
soma_retornosA=0
soma_retornosB=0

taxa_acerto=0
num_trades=0

fator_lucro=1
ftr_pos=1
ftr_neg=1

for stk in stocks:
    trades = pd.DataFrame(index=out_teste.index)
    trades['Close']=out_teste[stk].values

    short=melhor_short
    long=melhor_long
    
    trades['short'] = trades['Close'].rolling(window=short).mean()
    trades['long'] = trades['Close'].rolling(window=long).mean()
    
    trades['signalBuy']=0
    trades['signalSell']=0

 #   buy=0
 #   buyed=0
 #   sold=0
 #   
 #   for day in range(1,len(trades)):
 #       if sold==False:
 #           if (trades['short'][day]>trades['long'][day]):
 #               buy = 1  
 #           if buy == True:
 #               if (trades['short'][day]<trades['long'][day]):
 #                   buy = 0
 #               trades['signalBuy'][day]=buy
 #       
 #       if buy==False:
 #           if (trades['short'][day]<trades['long'][day]):
 #               sold = 1      
 #           if sold == True:
 #               if (trades['short'][day]>trades['long'][day]):
 #                   sold = 0
 #               trades['signalSell'][day]=sold
                
    buy=0
    buyed=0
    sold=0
    hold=False
    diasParado=0
    for day in range(1,len(trades)):
        if day>(diasParado+5):
            
            if sold==False & hold==False:
                if (trades['short'][day]>trades['long'][day]):
                    buy = 1  
                if buy == True:
                    if (trades['short'][day]<trades['long'][day]):
                        buy = 0
                        hold=True
                        diasParado=day
                    trades['signalBuy'][day]=buy

            if buy==False & hold==False:
                if (trades['short'][day]<trades['long'][day]):
                    sold = 1      
                if sold == True:
                    if (trades['short'][day]>trades['long'][day]):
                        sold = 0
                        hold=True
                        diasParado=day
                    trades['signalSell'][day]=sold
            hold=False
                
    #se estiver comprado no último dia vende

    if trades['signalBuy'][-1]==1 or trades['signalSell'][-1]==1:
        trades['signalBuy'][-1]=0
        trades['signalSell'][-1]=0

    # Posição dos trades
    trades['positionsBuy'] = trades['signalBuy'].diff()
    trades['positionsSell'] = trades['signalSell'].diff()
    
    fig = plt.subplots(figsize=(30,20))
    plt.style.use('seaborn-whitegrid')

    ax1 = plt.subplot(321)
    ax1.plot(trades['Close'], label='Close')
    ax1.plot(trades['short'], label='Short')
    ax1.plot(trades['long'],label='Long')

    ax1.plot(trades.loc[trades.positionsBuy == 1.0].index, 
             trades.short[trades.positionsBuy == 1.0],
             '^', markersize=10, color='m',label='Buy')
    ax1.plot(trades.loc[trades.positionsBuy == -1.0].index, 
             trades.short[trades.positionsBuy == -1.0],
             'v', markersize=10, color='k',label='Sell') 

    ax1.plot(trades.loc[trades.positionsSell == 1.0].index, 
             trades.short[trades.positionsSell == 1.0],
             '^', markersize=10, color='r',label='Sell')
    ax1.plot(trades.loc[trades.positionsSell == -1.0].index, 
             trades.short[trades.positionsSell == -1.0],
             'v', markersize=10, color='g',label='Buy') 


    ax1.set_ylabel("Preço [R$]",fontsize=25)
    ax1.set_xlabel("Data",fontsize=25)
    ax1.set_title("Médias" + " " + stk,fontsize=25)
    ax1.legend(loc='upper left',fontsize=25)

    ax5 = plt.subplot(323)
    #
    pos_buy_buyed=np.where(trades['positionsBuy']==1)
    pos_sell_buyed=trades['positionsBuy']==-1
    days_buy_buyed=trades['Close'][trades['positionsBuy']==1].index
    days_sell_buyed=trades['Close'][trades['positionsBuy']==-1].index
    distance_buyed=days_sell_buyed-days_buy_buyed
    #
    pos_buy_sold=np.where(trades['positionsSell']==1)
    pos_sell_sold=trades['positionsSell']==-1
    days_buy_sold=trades['Close'][trades['positionsSell']==1].index
    days_sell_sold=trades['Close'][trades['positionsSell']==-1].index
    distance_sold=days_sell_sold-days_buy_sold
    #
    buy_trade=[]
    sell_trade=[]
    # Gráfico para comprado
    for i in range(len(days_buy_buyed)):
        values_2=trades['Close'][trades['Close'].index==(days_buy_buyed[i]+distance_buyed[i])].values[0]
        values_1=trades['Close'][trades['Close'].index==days_buy_buyed[i]].values[0]
        buy_trade.append(100*(values_2-values_1)/values_1)

    valuesbuy = np.array(buy_trade)
    values1buy=buy_trade

    idx = np.array(list(range(len(buy_trade)))) 
    clrs = ['red' if (x < 0) else 'blue' for x in valuesbuy]
    sns.barplot(x=idx, y=valuesbuy, palette=clrs) 

    plt.ylabel('Retorno R[%] Comprado',fontsize=25)
    plt.xlabel('Trades Comprado',fontsize=25)

    ax52 = ax5.twinx()

    ax52.plot(sumper(valuesbuy),'gold',linewidth=4)

    plt.ylabel('Retorno Acumulado R[$]',fontsize=25)
    plt.xlabel('Trades',fontsize=25)
    plt.title('Retorno ' + " " + stk,fontsize=25)

    #Gráfico para vendido
    ax5segundo = plt.subplot(325)
    
    for i in range(len(days_buy_sold)):
        values_2=trades['Close'][trades['Close'].index==(days_buy_sold[i]+distance_sold[i])].values[0]
        values_1=trades['Close'][trades['Close'].index==days_buy_sold[i]].values[0]
        sell_trade.append(100*(values_1-values_2)/values_1)

    valuessell = np.array(sell_trade)
    values1sell=sell_trade

    idx = np.array(list(range(len(sell_trade)))) 
    clrs = ['red' if (x < 0) else 'blue' for x in valuessell]
    sns.barplot(x=idx, y=valuessell, palette=clrs) 

    plt.ylabel('Retorno R[%] Vendido',fontsize=25)
    plt.xlabel('Trades Vendido',fontsize=25)

    ax5segundo2 = ax5segundo.twinx()

    ax5segundo2.plot(sumper(valuessell),'gold',linewidth=4)

    plt.ylabel('Retorno Acumulado R[$]',fontsize=25)
    plt.xlabel('Trades',fontsize=25)
    plt.title('Retorno ' + " " + stk,fontsize=25)
    
    plt.tight_layout()
    
    a=1
    for kint in range(len(valuesbuy)):
        a*=((valuesbuy[kint]/100)+1)
        if valuesbuy[kint]>0:
            taxa_acerto+=1
            ftr_pos*=((valuesbuy[kint]/100)+1)
        else:
            ftr_neg*=((valuesbuy[kint]/-100)+1)
    soma_retornosA+=(a*CapitalInicial[i12])
    
    b=1
    for kint in range(len(valuessell)):
        b*=((valuessell[kint]/100)+1)
        if valuessell[kint]>0:
            taxa_acerto+=1
            ftr_pos*=((valuessell[kint]/100)+1)
        else:
            ftr_neg*=((valuessell[kint]/-100)+1)
    soma_retornosB+=(b*CapitalInicial[i12])
    
    #aux=((a*b)-1)
    aux=(a*b)
    
    patrimonio=[]
    patrimonio.append(CapitalInicial[i12]*100000)
    patrimonio.append(patrimonio[0]*aux)
    Trades[i12]=patrimonio
    
    num_trades+=(len(valuesbuy)+len(valuessell))
    
    i12+=1
    
taxa_acerto=(taxa_acerto/num_trades)*100

fator_lucro= ((ftr_pos-1)*100000)/((ftr_neg-1)*100000)

In [ ]:
num_trades

In [ ]:
print(round(fator_lucro,2))

In [ ]:
print("Taxa de acerto: {0}%" .format(round(taxa_acerto,2)))

In [ ]:
numTrades=[a for a in range(len(Trades[0]))]
irbr3=pd.DataFrame(index=numTrades)
irbr3['Patrimonio IRBR3']=Trades[0]
#irbr3.loc[-1]=[(CapitalInicial[0]*100000)]
#irbr3.index=irbr3.index+1
#irbr3=irbr3.sort_index()


numTrades=[a for a in range(len(Trades[1]))]
rent3=pd.DataFrame(index=numTrades)
rent3['Patrimonio RENT3']=Trades[1]
#rent3.loc[-1]=[(CapitalInicial[1]*100000)]
#rent3.index=rent3.index+1
#rent3=rent3.sort_index()

numTrades=[a for a in range(len(Trades[2]))]
mglu3=pd.DataFrame(index=numTrades)
mglu3['Patrimonio MGLU3']=Trades[2]
#mglu3.loc[-1]=[(CapitalInicial[2]*100000)]
#mglu3.index=mglu3.index+1
#mglu3=mglu3.sort_index()

numTrades=[a for a in range(len(Trades[3]))]
jbss3=pd.DataFrame(index=numTrades)
jbss3['Patrimonio JBSS3']=Trades[3]
#jbss3.loc[-1]=[(CapitalInicial[3]*100000)]
#jbss3.index=jbss3.index+1
#jbss3=jbss3.sort_index()

In [ ]:
#ax1 = plt.subplot(321)
irbr3.plot()
#ax2 = plt.subplot(322)
rent3.plot()
#ax3 = plt.subplot(323)
mglu3.plot()
#ax4 = plt.subplot(324)
jbss3.plot()

Patrimônio com a carteira ao final do perído

In [ ]:
patrimonio=0
for a in range(len(Trades)):
    patrimonio+=Trades[a][-1]
print("Patrimônio final: R$" + str(round(patrimonio,2)))

Com short==20 e long==57 Patrimônio final: R$106607.52

Com short==3 e long==58 Patrimônio final: R$137928.14

Com short==5 e long==69 Patrimônio final: R$125335.67

In [ ]:
melhor_short=3
melhor_long=58

# Questão 2
### Atribuir valores aleatórios para as médias short e long para otimizar e escolher as métricas que possuem maior rendimento apenas em trades comprados, e usá-las para otimizar bandas de boolinger com estratégia " Walking up the bands", seguidor de tendência, apenas para comprado também.

In [ ]:
out_otim

In [ ]:
Retornos=[]
retornos_aux=[]
melhor_short_comprado=0
melhor_long_comprado=0
melhor_retorno=0
intervalo_short=15-1+1
intervalo_long=45-16+1
valores_short=[]
valores_long=[]
for iteracao in tqdm(range(1,(intervalo_short*intervalo_long+1))):
    
    if (iteracao/intervalo_long) == int(iteracao/intervalo_long):
        short=int(iteracao/intervalo_long)
    else:
        short=int(iteracao/intervalo_long)+1
    
    long=iteracao-(intervalo_long*(short-1))+intervalo_short
    
    soma_retornos=0
    i2=0
    for stk in stocks:
        trades = pd.DataFrame(index=out_otim.index)
        trades['Close']=out_otim[stk].values


        trades['short'] = trades['Close'].rolling(window=short).mean()
        trades['long'] = trades['Close'].rolling(window=long).mean()

        # Sinal de Entrada
        trades['signal']=0
        trades['signal'][short:] = np.where(trades['short'][short:] > trades['long'][short:], 1.0, 0.0)   

        if trades['signal'][-1]==1:
            trades['signal'][-1]=0
        # Posição dos trades
        trades['positions'] = trades['signal'].diff()

        pos_buy=np.where(trades['positions']==1)
        pos_sell=trades['positions']==-1
        days_buy=trades['Close'][trades['positions']==1].index
        days_sell=trades['Close'][trades['positions']==-1].index
        distance=days_sell-days_buy
        buy_trade=[]
        for i in range(len(days_buy)):
            values_2=trades['Close'][trades['Close'].index==(days_buy[i]+distance[i])].values[0]
            values_1=trades['Close'][trades['Close'].index==days_buy[i]].values[0]
            buy_trade.append(100*(values_2-values_1)/values_1)

        values = np.array(buy_trade)
        values1=buy_trade
        
        #soma_retornos+=(values.sum())*(CapitalInicial[i2])
        
        a=1
        for kint in range(len(values)):
            a*=((values[kint]/100)+1)
        a=CapitalInicial[i2]*(a-1)
        soma_retornos+=a
                             
        i2+=1
        
    #if iteracao==0:
     #   Retornos.append(-1000)
    
    Retornos.append(soma_retornos*100)
    valores_short.append(short)
    valores_long.append(long)
    retornos_aux=sorted(Retornos,reverse=True)
    
    if retornos_aux[0]==soma_retornos*100:
        melhor_short_comprado=short
        melhor_long_comprado=long
        melhor_retorno=soma_retornos*100
    
    
print(melhor_short_comprado,melhor_long_comprado,melhor_retorno)

# Gráfico com as métricas obtidas na otimização

In [ ]:
taxa_acerto=0
num_trades=0

fator_lucro=1
ftr_pos=1
ftr_neg=1

for stk in stocks:
    trades = pd.DataFrame(index=out_otim.index)
    trades['Close']=out_otim[stk].values

    short=melhor_short_comprado
    long=melhor_long_comprado

    trades['short'] = trades['Close'].rolling(window=short).mean()
    trades['long'] = trades['Close'].rolling(window=long).mean()

    # Sinal de Entrada
    trades['signal']=0
    trades['signal'][short:] = np.where(trades['short'][short:] > trades['long'][short:], 1.0, 0.0)   

    if trades['signal'][-1]==1:
        trades['signal'][-1]=0
    # Posição dos trades
    trades['positions'] = trades['signal'].diff()
    
    fig = plt.subplots(figsize=(20,15))
    plt.style.use('seaborn-whitegrid')

    ax1 = plt.subplot(321)
    ax1.plot(trades['Close'], label='Close')
    ax1.plot(trades['short'], label='Short')
    ax1.plot(trades['long'],label='Long')

    ax1.plot(trades.loc[trades.positions == 1.0].index, 
             trades.short[trades.positions == 1.0],
             '^', markersize=10, color='m',label='Buy')
    ax1.plot(trades.loc[trades.positions == -1.0].index, 
             trades.short[trades.positions == -1.0],
             'v', markersize=10, color='k',label='Sell') 

    ax1.set_ylabel("Preço [R$]",fontsize=25)
    ax1.set_xlabel("Data",fontsize=25)
    ax1.set_title("Médias" + " " + stk,fontsize=25)
    ax1.legend(loc='upper left',fontsize=25)

    ax5 = plt.subplot(323)
    pos_buy=np.where(trades['positions']==1)
    pos_sell=trades['positions']==-1
    days_buy=trades['Close'][trades['positions']==1].index
    days_sell=trades['Close'][trades['positions']==-1].index
    distance=days_sell-days_buy
    buy_trade=[]
    for i in range(len(days_buy)):
        values_2=trades['Close'][trades['Close'].index==(days_buy[i]+distance[i])].values[0]
        values_1=trades['Close'][trades['Close'].index==days_buy[i]].values[0]
        buy_trade.append(100*(values_2-values_1)/values_1)

    values = np.array(buy_trade)
    values1=buy_trade

    idx = np.array(list(range(len(buy_trade)))) 
    clrs = ['red' if (x < 0) else 'blue' for x in values]
    sns.barplot(x=idx, y=values, palette=clrs) 

    plt.ylabel('Retorno R[%]',fontsize=25)
    plt.xlabel('Trades',fontsize=25)

    ax52 = ax5.twinx()

    ax52.plot(sumper(values),'gold',linewidth=4)

    plt.ylabel('Retorno Acumulado R[$]',fontsize=25)
    plt.xlabel('Trades',fontsize=25)
    plt.title('Retorno ' + " " + stk,fontsize=25)


    ax6 = plt.subplot(324)
    pos_buy=np.where(trades['positions']==1)
    pos_sell=trades['positions']==-1
    days_buy=trades['Close'][trades['positions']==1].index
    days_sell=trades['Close'][trades['positions']==-1].index
    distance=days_sell-days_buy
    buy_trade=[]
    for i in range(len(days_buy)):
        values_2=trades['Close'][trades['Close'].index==(days_buy[i]+distance[i])].values[0]
        values_1=trades['Close'][trades['Close'].index==days_buy[i]].values[0]
        buy_trade.append(100*(values_2-values_1)/values_1)
    #values = np.array(buy_trade[-6:])   
    idx = np.array(list(range(len(buy_trade)))) 
    clrs = ['red' if (x < 0) else 'blue' for x in values]
    sns.barplot(x=idx, y=values, palette=clrs) 


    plt.ylabel('Retorno [%]',fontsize=25)
    plt.xlabel('Trades',fontsize=25)
    plt.title('Retorno ' + " " + stk,fontsize=25)
    plt.tight_layout()
    
    for iteracao in range(len(values)):
        if values[iteracao]>0:
            taxa_acerto+=1
            ftr_pos*=((values[iteracao]/100)+1)
        else:
            ftr_neg*=((values[iteracao]/-100)+1)

    num_trades+=len(values)

taxa_acerto=(taxa_acerto/num_trades)*100

fator_lucro= ((ftr_pos-1)*100000)/((ftr_neg-1)*100000)

In [ ]:
print(round(fator_lucro,2))

In [ ]:
print("Taxa de acerto: {0}%" .format(round(taxa_acerto,2)))

Patrimônio ao final do periodo 

In [ ]:
((melhor_retorno/100)+1)*100000

# Otimizar bandas de bollinger na estratégia walking up the bands com long e short obtidos acima

In [ ]:
def bollinger_bands(data, trend_periods, deviation, close_col='Close'):

    data['bol_bands_middle'] = data[close_col].ewm(ignore_na=False, min_periods=0, com=trend_periods, adjust=True).mean()
    i=0
    for index, row in data.iterrows():

        s = data[close_col].iloc[i - trend_periods: i]
        sums = 0
        middle_band = data.at[index, 'bol_bands_middle']
        for e in s:
            sums += np.square(e - middle_band)

        std = np.sqrt(sums / trend_periods)
   #     d = 2
        upper_band = middle_band + (deviation * std)
        lower_band = middle_band - (deviation * std)

        #data.set_value(index, 'bol_bands_upper', upper_band)
        data.at[index, 'bol_bands_upper'] = upper_band
        #data.set_value(index, 'bol_bands_lower', lower_band)
        data.at[index, 'bol_bands_lower'] = lower_band
        
        i+=1

    return data


In [ ]:
Retornos=[]
intervalo_trend=45-5+1
intervalo_deviation=6-1+1
valores_trend=[]
valores_deviation=[]

for iteracao in tqdm(range(1,(intervalo_trend*intervalo_deviation)+1)):
    
    if (iteracao/intervalo_trend) == int(iteracao/intervalo_trend):
        deviation=int(iteracao/intervalo_trend)
    else:
        deviation=int(iteracao/intervalo_trend)+1
    
    trend_periods=iteracao-(intervalo_trend*(deviation-1))+(5-1) #menor valor de trend - 1
    
    soma_retornos=0
    i2=0
    for stk in stocks:
        
        short=melhor_short_comprado
        long=melhor_long_comprado
        
        trades = pd.DataFrame(index=out_otim.index)
        trades['Close']=out_otim[stk].values

        trades.reset_index(inplace=True)
        bollinger_bands(trades, trend_periods, deviation)

        trades['short'] = trades['Close'].rolling(window=short).mean()
        trades['long'] = trades['Close'].rolling(window=long).mean()

        trades['signal']=0
        #trades['signal'][short:] = np.where(trades['short'][short:] > trades['long'][short:], 1.0, 0.0) 

        trades=trades.set_index('Date')
        # Sinal de Entrada
        #trades['signal']=0
        signal=np.zeros(len(trades['signal']))
        buy=0
        for day in range(1,len(trades)):
            cond1_buy=trades['Close'].iloc[day-1] < trades['bol_bands_upper'].iloc[day-1]
            cond2_buy=trades['Close'].iloc[day] > trades['bol_bands_upper'].iloc[day]  
            cond1_sell=trades['Close'].iloc[day-1] > trades['bol_bands_lower'].iloc[day-1]
            cond2_sell=trades['Close'].iloc[day] < trades['bol_bands_lower'].iloc[day]

            if (cond1_buy & cond2_buy) | (trades['short'][day]>trades['long'][day]):
                buy = 1
            if buy == True:
                if (cond1_sell & cond2_sell) | (trades['short'][day]<trades['long'][day]):
                    buy = 0
              #  if cond1_sell & cond2_sell:
              #      buy = 0
              #  if (trades['short'][day]<trades['long'][day]):
              #      buy = 0
            trades['signal'][day]=buy
        
        #se estiver comprado no último dia vende

        if trades['signal'][-1]==1:
            trades['signal'][-1]=0

        # Posição dos trades
        trades['positions'] = trades['signal'].diff()
        
        pos_buy=np.where(trades['positions']==1)
        pos_sell=trades['positions']==-1
        days_buy=trades['Close'][trades['positions']==1].index
        days_sell=trades['Close'][trades['positions']==-1].index
        distance=days_sell-days_buy
        buy_trade=[]
        for i in range(len(days_buy)):
            values_2=trades['Close'][trades['Close'].index==(days_buy[i]+distance[i])].values[0]
            values_1=trades['Close'][trades['Close'].index==days_buy[i]].values[0]
            buy_trade.append(100*(values_2-values_1)/values_1)
        values = np.array(buy_trade)   
        values3=buy_trade

        
        #soma_retornos+=(values.sum())*(CapitalInicial[i2])
        a=1
        for kint in range(len(values)):
            a*=((values[kint]/100)+1)
        a=CapitalInicial[i2]*(a-1)
        soma_retornos+=a
                            
                             
        i2+=1
        
    #if iteracao==0:
    #    Retornos.append(-1000)
    
    Retornos.append(soma_retornos*100)
    valores_trend.append(trend_periods)
    valores_deviation.append(deviation)
    retornos_aux=sorted(Retornos,reverse=True)
    
    if retornos_aux[0]==soma_retornos*100:
        melhor_trend_periods=trend_periods
        melhor_deviation=deviation
        melhor_retorno=soma_retornos*100
    
    
print(melhor_trend_periods,melhor_deviation,(melhor_retorno))

# Gráficos com as métricas obtidas na otimização

In [ ]:
taxa_acerto=0
num_trades=0

fator_lucro=1
ftr_pos=1
ftr_neg=1

for stk in stocks:    
    trades = pd.DataFrame(index=out_otim.index)
    trades['Close']=out_otim[stk].values

    trades.reset_index(inplace=True)
    bollinger_bands(trades, melhor_trend_periods, melhor_deviation)

    short=melhor_short_comprado
    long=melhor_long_comprado

    trades['short'] = trades['Close'].rolling(window=short).mean()
    trades['long'] = trades['Close'].rolling(window=long).mean()

    trades['signal']=0
    #trades['signal'][short:] = np.where(trades['short'][short:] > trades['long'][short:], 1.0, 0.0) 

    trades=trades.set_index('Date')
    # Sinal de Entrada
    #trades['signal']=0
    signal=np.zeros(len(trades['signal']))
    buy=0
    for day in range(1,len(trades)):
        cond1_buy=trades['Close'].iloc[day-1] < trades['bol_bands_upper'].iloc[day-1]
        cond2_buy=trades['Close'].iloc[day] > trades['bol_bands_upper'].iloc[day]  
        cond1_sell=trades['Close'].iloc[day-1] > trades['bol_bands_lower'].iloc[day-1]
        cond2_sell=trades['Close'].iloc[day] < trades['bol_bands_lower'].iloc[day]

        if (cond1_buy & cond2_buy) & (trades['short'][day]>trades['long'][day]):
            buy = 1
        if buy == True:
            if (cond1_sell & cond2_sell) | (trades['short'][day]<trades['long'][day]):
                buy = 0
            #if (trades['short'][day]<trades['long'][day]):
                #buy = 0
        trades['signal'][day]=buy
    #se estiver comprado no último dia vende

    if trades['signal'][-2]==1:
        trades['signal'][-1]=0
    #trades['signal']=signal
    #condition2=trades['Close'] < trades['bol_bands_lower']

    #trades['signal'] = np.where(, 1.0, 0.0)   

    #trades
    # Posição dos trades
    trades['positions'] = trades['signal'].diff()

    fig = plt.subplots(figsize=(20,20))
    plt.style.use('seaborn-whitegrid')

    ax1 = plt.subplot(311)
    ax1.plot( trades.Close, 'y', label = 'Close')
    ax1.plot( trades.bol_bands_middle, 'k:', label = 'middle')
    ax1.plot( trades.bol_bands_upper, 'm', label = 'upper')
    ax1.plot( trades.bol_bands_lower, 'g', label = 'lower')
    ax1.plot(trades['short'][-494:], label='Short')
    ax1.plot(trades['long'][-494:],label='Long')

    ax1.plot(trades.loc[trades.positions == 1.0].index, 
             trades.bol_bands_upper[trades.positions == 1.0],
             '^', markersize=10, color='m',label='Buy')
    ax1.plot(trades.loc[trades.positions == -1.0].index, 
             trades.bol_bands_lower[trades.positions == -1.0],
             'v', markersize=10, color='k',label='Sell') 

    ax1.set_ylabel("Preço [R$]",fontsize=25)
    ax1.set_xlabel("Data",fontsize=25)
    ax1.set_title("Médias " + stk,fontsize=25)
    ax1.legend(loc='upper left',fontsize=25)

    trades['Close_recorte']=trades['Close']*trades['signal']

    #trades['Close_recorte'][trades['Close_recorte']==0]=16

    ax3 = plt.subplot(312)
    ax3.plot(trades['Close_recorte'], label='Close')
    ax3.set_ylabel("Preço [R$]",fontsize=25)
    ax3.set_xlabel("Data",fontsize=25)
    ax3.set_title("Médias " + stk,fontsize=25)
    ax3.legend(loc='upper left',fontsize=25)
    

    ax5 = plt.subplot(313)
    pos_buy=np.where(trades['positions']==1)
    pos_sell=trades['positions']==-1
    days_buy=trades['Close'][trades['positions']==1].index
    days_sell=trades['Close'][trades['positions']==-1].index
    distance=days_sell-days_buy
    buy_trade=[]
    for i in range(len(days_buy)):
        values_2=trades['Close'][trades['Close'].index==(days_buy[i]+distance[i])].values[0]
        values_1=trades['Close'][trades['Close'].index==days_buy[i]].values[0]
        buy_trade.append(100*(values_2-values_1)/values_1)
    values = np.array(buy_trade)   
    values3=buy_trade
    plt.ylabel('Retorno [%]',fontsize=25)
    plt.xlabel('Trades',fontsize=25)
    plt.title("Retornos " + stk,fontsize=25)


    idx = np.array(list(range(len(buy_trade)))) 
    clrs = ['red' if (x < 0) else 'blue' for x in values]
    sns.barplot(x=idx, y=values, palette=clrs) 
    ax52 = ax5.twinx()
    ax52.plot(sumper(values),'gold',linewidth=4)

    plt.ylabel('Retorno Acumulado [%]',fontsize=25)
    plt.xlabel('Trades',fontsize=25)
    plt.title("Retornos " + stk,fontsize=25)

    plt.tight_layout()
    
    for iteracao in range(len(values)):
        if values[iteracao]>0:
            taxa_acerto+=1
            ftr_pos*=((values[iteracao]/100)+1)
        else:
            ftr_neg*=((values[iteracao]/-100)+1)

    num_trades+=len(values)

taxa_acerto=(taxa_acerto/num_trades)*100

fator_lucro= ((ftr_pos-1)*100000)/((ftr_neg-1)*100000)

In [ ]:
print(round(fator_lucro,2))

In [ ]:
print("Taxa de acerto: {0}%" .format(round(taxa_acerto,2)))

Patrimônio ao final do periodo 

In [ ]:
((melhor_retorno/100)+1)*100000

# Aplicando no período teste

In [ ]:
out_teste

In [ ]:
Trades=[]
Trades.append(0)
Trades.append(0)
Trades.append(0)
Trades.append(0)

taxa_acerto=0
num_trades=0
i12=0

fator_lucro=1
ftr_pos=1
ftr_neg=1

#Trades=pd.DataFrame(columns=stocks)
for stk in stocks:    
    trades = pd.DataFrame(index=out_teste.index)
    trades['Close']=out_teste[stk].values

    trades.reset_index(inplace=True)
    bollinger_bands(trades, melhor_trend_periods, melhor_deviation)

    short=melhor_short_comprado
    long=melhor_long_comprado

    trades['short'] = trades['Close'].rolling(window=short).mean()
    trades['long'] = trades['Close'].rolling(window=long).mean()

    trades['signal']=0
    #trades['signal'][short:] = np.where(trades['short'][short:] > trades['long'][short:], 1.0, 0.0) 

    trades=trades.set_index('Date')
    # Sinal de Entrada
    #trades['signal']=0
    signal=np.zeros(len(trades['signal']))
    buy=0
    for day in range(1,len(trades)):
        cond1_buy=trades['Close'].iloc[day-1] < trades['bol_bands_upper'].iloc[day-1]
        cond2_buy=trades['Close'].iloc[day] > trades['bol_bands_upper'].iloc[day]  
        cond1_sell=trades['Close'].iloc[day-1] > trades['bol_bands_lower'].iloc[day-1]
        cond2_sell=trades['Close'].iloc[day] < trades['bol_bands_lower'].iloc[day]

        if (cond1_buy & cond2_buy) | (trades['short'][day]>trades['long'][day]):
            buy = 1
        if buy == True:
            if (cond1_sell & cond2_sell) & (trades['short'][day]<trades['long'][day]):
                buy = 0
            
         #   if cond1_sell & cond2_sell:
         #       buy = 0
         #   if (trades['short'][day]<trades['long'][day]):
         #       buy = 0
        trades['signal'][day]=buy
    #se estiver comprado no último dia vende

    if trades['signal'][-2]==1:
        trades['signal'][-1]=0
    #trades['signal']=signal
    #condition2=trades['Close'] < trades['bol_bands_lower']

    #trades['signal'] = np.where(, 1.0, 0.0)   

    #trades
    # Posição dos trades
    trades['positions'] = trades['signal'].diff()

    fig = plt.subplots(figsize=(20,20))
    plt.style.use('seaborn-whitegrid')

    ax1 = plt.subplot(211)
    ax1.plot( trades.Close, 'y', label = 'Close')
    ax1.plot( trades.bol_bands_middle, 'k:', label = 'middle')
    ax1.plot( trades.bol_bands_upper, 'm', label = 'upper')
    ax1.plot( trades.bol_bands_lower, 'g', label = 'lower')
    ax1.plot(trades['short'][-247:], label='Short')
    ax1.plot(trades['long'][-247:],label='Long')

    ax1.plot(trades.loc[trades.positions == 1.0].index, 
             trades.bol_bands_lower[trades.positions == 1.0],
             '^', markersize=10, color='m',label='Buy')
    ax1.plot(trades.loc[trades.positions == -1.0].index, 
             trades.bol_bands_middle[trades.positions == -1.0],
             'v', markersize=10, color='k',label='Sell') 

    ax1.set_ylabel("Preço [R$]",fontsize=25)
    ax1.set_xlabel("Data",fontsize=25)
    ax1.set_title("Médias " + stk,fontsize=25)
    ax1.legend(loc='upper left',fontsize=25)



    trades['Close_recorte']=trades['Close']*trades['signal']

    #trades['Close_recorte'][trades['Close_recorte']==0]=16

    ax3 = plt.subplot(413)
    ax3.plot(trades['Close_recorte'], label='Close')
    ax3.set_ylabel("Preço [R$]",fontsize=25)
    ax3.set_xlabel("Data",fontsize=25)
    ax3.set_title("Médias " + stk,fontsize=25)
    ax3.legend(loc='upper left',fontsize=25)
    

    ax5 = plt.subplot(414)
    pos_buy=np.where(trades['positions']==1)
    pos_sell=trades['positions']==-1
    days_buy=trades['Close'][trades['positions']==1].index
    days_sell=trades['Close'][trades['positions']==-1].index
    distance=days_sell-days_buy
    buy_trade=[]
    for i in range(len(days_buy)):
        values_2=trades['Close'][trades['Close'].index==(days_buy[i]+distance[i])].values[0]
        values_1=trades['Close'][trades['Close'].index==days_buy[i]].values[0]
        buy_trade.append(100*(values_2-values_1)/values_1)
    values = np.array(buy_trade)   
    values3=buy_trade
    plt.ylabel('Retorno [%]',fontsize=25)
    plt.xlabel('Trades',fontsize=25)
    plt.title("Retornos " + stk,fontsize=25)


    idx = np.array(list(range(len(buy_trade)))) 
    clrs = ['red' if (x < 0) else 'blue' for x in values]
    sns.barplot(x=idx, y=values, palette=clrs) 
    ax52 = ax5.twinx()
    ax52.plot(sumper(values),'gold',linewidth=4)

    plt.ylabel('Retorno Acumulado [%]',fontsize=25)
    plt.xlabel('Trades',fontsize=25)
    plt.title("Retornos " + stk,fontsize=25)
    
    patrimonio=[]
    patrimonio.append(CapitalInicial[i12]*100000)
    for k in range(1,len(values)+1):
        patrimonio.append(patrimonio[k-1]*(1+(values[k-1]/100)))
        if values[k-1]>0:
            taxa_acerto+=1
            ftr_pos*=((values[k-1]/100)+1)
        else:
            ftr_neg*=((values[k-1]/-100)+1)
  
    Trades[i12]=patrimonio
    
    num_trades+=len(values)
    
    
    i12=i12+1

    plt.tight_layout()
    
taxa_acerto=(taxa_acerto/num_trades)*100

fator_lucro= ((ftr_pos-1)*100000)/((ftr_neg-1)*100000)

In [ ]:
num_trades

In [ ]:
print(round(fator_lucro,2))

In [ ]:
print("Taxa de acerto: {0}%" .format(round(taxa_acerto,2)))

In [ ]:
numTrades=[a for a in range(len(Trades[0]))]
irbr3=pd.DataFrame(index=numTrades)
irbr3['Patrimonio IRBR3']=Trades[0]

numTrades=[a for a in range(len(Trades[1]))]
rent3=pd.DataFrame(index=numTrades)
rent3['Patrimonio RENT3']=Trades[1]

numTrades=[a for a in range(len(Trades[2]))]
mglu3=pd.DataFrame(index=numTrades)
mglu3['Patrimonio MGLU3']=Trades[2]

numTrades=[a for a in range(len(Trades[3]))]
jbss3=pd.DataFrame(index=numTrades)
jbss3['Patrimonio JBSS3']=Trades[3]

In [ ]:
#ax1 = plt.subplot(321)
irbr3.plot()
#ax2 = plt.subplot(322)
rent3.plot()
#ax3 = plt.subplot(323)
mglu3.plot()
#ax4 = plt.subplot(324)
jbss3.plot()

In [ ]:
patrimonio=0
for a in range(len(Trades)):
    patrimonio+=Trades[a][-1]
print("Patrimônio final: R$" + str(round(patrimonio,2)))

# Questão 3

In [ ]:
out_otim

In [ ]:
def bollinger_bands(data, trend_periods, deviation, close_col='Close'):

    data['bol_bands_middle'] = data[close_col].ewm(ignore_na=False, min_periods=0, com=trend_periods, adjust=True).mean()
    i=0
    for index, row in data.iterrows():

        s = data[close_col].iloc[i - trend_periods: i]
        sums = 0
        middle_band = data.at[index, 'bol_bands_middle']
        for e in s:
            sums += np.square(e - middle_band)

        std = np.sqrt(sums / trend_periods)
   #     d = 2
        upper_band = middle_band + (deviation * std)
        lower_band = middle_band - (deviation * std)

        #data.set_value(index, 'bol_bands_upper', upper_band)
        data.at[index, 'bol_bands_upper'] = upper_band
        #data.set_value(index, 'bol_bands_lower', lower_band)
        data.at[index, 'bol_bands_lower'] = lower_band
        
        i+=1

    return data


# Será otimizado usando médias móveis adquiridas anteriormente e bandas de bollinger

In [ ]:
print(melhor_short, melhor_long)

## Operar comprado e vendido

In [ ]:
Retornos=[]
intervalo_trend=30-2+1
intervalo_deviation=6-1+1
valores_trend=[]
valores_deviation=[]

for iteracao in tqdm(range(1,(intervalo_trend*intervalo_deviation)+1)):
    
    if (iteracao/intervalo_trend) == int(iteracao/intervalo_trend):
        deviation=int(iteracao/intervalo_trend)
    else:
        deviation=int(iteracao/intervalo_trend)+1
    
    trend_periods=iteracao-(intervalo_trend*(deviation-1))+(5-1) #menor valor de trend - 1
    
    soma_retornos=0
    soma_retornosComprado=0
    soma_retornosVendido=0
    i2=0
    for stk in stocks:
        
        short=melhor_short
        long=melhor_long
        
        trades = pd.DataFrame(index=out_otim.index)
        trades['Close']=out_otim[stk].values

        trades.reset_index(inplace=True)
        bollinger_bands(trades, trend_periods, deviation)

        trades['short'] = trades['Close'].rolling(window=short).mean()
        trades['long'] = trades['Close'].rolling(window=long).mean()
        
        #
        trades['signalBuy']=0
        trades['signalSell']=0
        #
        
        #trades['signal'][short:] = np.where(trades['short'][short:] > trades['long'][short:], 1.0, 0.0) 

        #trades=trades.set_index('Date')
        # Sinal de Entrada
        #trades['signal']=0
        #signal=np.zeros(len(trades['signal']))
        
    
        buy=0
        buyed=0
        sold=0
        hold=False
        diasParado=0
        for day in range(1,len(trades)):
            if day>(diasParado+5):
                cond1_buy=trades['Close'].iloc[day-1] > trades['bol_bands_lower'].iloc[day-1]
                cond2_buy=trades['Close'].iloc[day] < trades['bol_bands_lower'].iloc[day]  
                cond1_sell=trades['Close'].iloc[day-1] < trades['bol_bands_middle'].iloc[day-1]
                cond2_sell=trades['Close'].iloc[day] > trades['bol_bands_middle'].iloc[day]
                
                if sold==False & hold==False:
                    if (cond1_buy & cond2_buy) | (trades['short'][day]>trades['long'][day]):
                        buy = 1  
                    if buy == True:
                        if (cond1_sell & cond2_sell) | (trades['short'][day]<trades['long'][day]):
                            buy = 0
                            hold=True
                            diasParado=day
                        trades['signalBuy'][day]=buy

                if buy==False & hold==False:
                    if (cond1_sell & cond2_sell) | (trades['short'][day]<trades['long'][day]):
                        sold = 1      
                    if sold == True:
                        if (cond1_buy & cond2_buy) | (trades['short'][day]>trades['long'][day]):
                            sold = 0
                            hold=True
                            diasParado=day
                        trades['signalSell'][day]=sold
                hold=False
            
            
#        buyed=0
#        sold=0
#        buy=0    
#            
#        for day in range(1,len(trades)):
#            cond1_buy=trades['Close'].iloc[day-1] > trades['bol_bands_lower'].iloc[day-1]
#            cond2_buy=trades['Close'].iloc[day] < trades['bol_bands_lower'].iloc[day]  
#            cond1_sell=trades['Close'].iloc[day-1] < trades['bol_bands_middle'].iloc[day-1]
#            cond2_sell=trades['Close'].iloc[day] > trades['bol_bands_middle'].iloc[day]
#            
#            if sold==False:
#                if (cond1_buy & cond2_buy) | (trades['short'][day]>trades['long'][day]):
#                    buy = 1
#                if buy == True:
#                    if (cond1_sell & cond2_sell) & (trades['short'][day]<trades['long'][day]):
#                        buy = 0
#                    trades['signalBuy'][day]=buy
#
#            if buy==False:
#                if (cond1_sell & cond2_sell) | (trades['short'][day]<trades['long'][day]):
#                    sold = 1
#                if sold==True:
#                    if (cond1_buy & cond2_buy) & (trades['short'][day]>trades['long'][day]):
#                        sold=0
#                    trades['signalSell'][day]=sold
                    
        #se estiver comprado no último dia vende

        if trades['signalBuy'][len(trades['signalBuy'])-1]==1 or trades['signalSell'][len(trades['signalSell'])-1]==1:
            trades['signalBuy'][len(trades['signalBuy'])-1]=0
            trades['signalSell'][len(trades['signalSell'])-1]=0

        # Posição dos trades
        trades['positionsBuy'] = trades['signalBuy'].diff()
        trades['positionsSell'] = trades['signalSell'].diff()
    
        pos_buy_buyed=np.where(trades['positionsBuy']==1)
        pos_sell_buyed=trades['positionsBuy']==-1
        days_buy_buyed=trades['Close'][trades['positionsBuy']==1].index
        days_sell_buyed=trades['Close'][trades['positionsBuy']==-1].index
        distance_buyed=days_sell_buyed-days_buy_buyed
        #
        pos_buy_sold=np.where(trades['positionsSell']==1)
        pos_sell_sold=trades['positionsSell']==-1
        days_buy_sold=trades['Close'][trades['positionsSell']==1].index
        days_sell_sold=trades['Close'][trades['positionsSell']==-1].index
        distance_sold=days_sell_sold-days_buy_sold
        #
        buy_trade=[]
        sell_trade=[]
        
        # Gráfico para comprado
        for i in range(len(days_buy_buyed)):
            values_2=trades['Close'][trades['Close'].index==(days_buy_buyed[i]+distance_buyed[i])].values[0]
            values_1=trades['Close'][trades['Close'].index==days_buy_buyed[i]].values[0]
            buy_trade.append(100*(values_2-values_1)/values_1)

        valuesbuy = np.array(buy_trade)
        values1buy=buy_trade
        
        #Gráfico para vendido
        for i in range(len(days_buy_sold)):
            values_2=trades['Close'][trades['Close'].index==(days_buy_sold[i]+distance_sold[i])].values[0]
            values_1=trades['Close'][trades['Close'].index==days_buy_sold[i]].values[0]
            sell_trade.append(100*(values_1-values_2)/values_1)

        valuessell = np.array(sell_trade)
        values1sell=sell_trade

        
        a=1
        for kint in range(len(valuesbuy)):
            a*=((valuesbuy[kint]/100)+1)
        soma_retornosComprado+=(a*CapitalInicial[i2])

        b=1
        for kint in range(len(valuessell)):
            b*=((valuessell[kint]/100)+1)
        soma_retornosVendido+=(b*CapitalInicial[i2])

        soma_retornos+=((soma_retornosComprado*soma_retornosVendido)-1)

        i2+=1
        
    #if iteracao==0:
     #   Retornos.append(-1000)
    
    Retornos.append(soma_retornos*100)
    valores_trend.append(trend_periods)
    valores_deviation.append(deviation)
    retornos_aux=sorted(Retornos,reverse=True)
    
    if retornos_aux[0]==(soma_retornos*100):
        melhor_trend_periods=trend_periods
        melhor_deviation=deviation
        melhor_retorno=soma_retornos*100
    
    
print(melhor_trend_periods,melhor_deviation,melhor_retorno)

# Gráfico com Trend_period entre 5 e 30; deviation entre 1 e 6

In [ ]:
fig = plt.subplots(figsize=(40,20))
plt.style.use('seaborn-whitegrid')

ax1 = plt.subplot(311)
ax1.plot(valores_trend, label='trend_period')
ax1.plot(valores_deviation, label='deviation')
ax1.plot(Retornos, label='Retorno')
ax1.set_ylabel("Retorno [%]",fontsize=25)
ax1.set_xlabel("Número de testes",fontsize=25)
titulo="Maior retorno " + str(round(melhor_retorno,2)) + "%, " + "trend_periods: " + str(melhor_trend_periods) + " melhor_deviation: " + str(melhor_deviation)+ " Short: " + str(melhor_short) + " Long: " + str(melhor_long)
ax1.set_title(titulo,fontsize=25)

ax2 = plt.subplot(312)
sns.barplot(x=valores_trend, y=Retornos) 
ax2.set_ylabel("Retorno [%]",fontsize=25)
ax2.set_xlabel("Valores trend_period", fontsize=25)

ax3 = plt.subplot(313)
sns.barplot(x=valores_deviation, y=Retornos) 
ax3.set_ylabel("Retorno [%]",fontsize=25)
ax3.set_xlabel("Valores deviation", fontsize=25)

# Gráfico com Trend_period entre 2 e 30; deviation entre 1 e 6

In [ ]:
fig = plt.subplots(figsize=(40,20))
plt.style.use('seaborn-whitegrid')

ax1 = plt.subplot(311)
ax1.plot(valores_trend, label='trend_period')
ax1.plot(valores_deviation, label='deviation')
ax1.plot(Retornos, label='Retorno')
ax1.set_ylabel("Retorno [%]",fontsize=25)
ax1.set_xlabel("Número de testes",fontsize=25)
titulo="Maior retorno " + str(round(melhor_retorno,2)) + "%, " + "trend_periods: " + str(melhor_trend_periods) + " melhor_deviation: " + str(melhor_deviation)+ " Short: " + str(melhor_short) + " Long: " + str(melhor_long)
ax1.set_title(titulo,fontsize=25)

ax2 = plt.subplot(312)
sns.barplot(x=valores_trend, y=Retornos) 
ax2.set_ylabel("Retorno [%]",fontsize=25)
ax2.set_xlabel("Valores trend_period", fontsize=25)

ax3 = plt.subplot(313)
sns.barplot(x=valores_deviation, y=Retornos) 
ax3.set_ylabel("Retorno [%]",fontsize=25)
ax3.set_xlabel("Valores deviation", fontsize=25)

# Gráfico com as métricas obtidas na otimização

## Operar comprado e vendido

In [ ]:
i12=0
soma_retornosA=0
soma_retornosB=0
aux=[]

fator_lucro=1
ftr_pos=1
ftr_neg=1

taxa_acerto=0
num_trades=0

for stk in stocks:    
    short=melhor_short
    long=melhor_long

    trades = pd.DataFrame(index=out_otim.index)
    trades['Close']=out_otim[stk].values

    trades.reset_index(inplace=True)
    bollinger_bands(trades, melhor_trend_periods, melhor_deviation)

    trades['short'] = trades['Close'].rolling(window=short).mean()
    trades['long'] = trades['Close'].rolling(window=long).mean()

    trades['signalBuy']=0
    trades['signalSell']=0

    buy=0
    buyed=0
    sold=0
    hold=False
    diasParado=0
    for day in range(1,len(trades)):
        if day>(diasParado+5):
            cond1_buy=trades['Close'].iloc[day-1] > trades['bol_bands_lower'].iloc[day-1]
            cond2_buy=trades['Close'].iloc[day] < trades['bol_bands_lower'].iloc[day]  
            cond1_sell=trades['Close'].iloc[day-1] < trades['bol_bands_middle'].iloc[day-1]
            cond2_sell=trades['Close'].iloc[day] > trades['bol_bands_middle'].iloc[day]

            if sold==False & hold==False:
                if (cond1_buy & cond2_buy) | (trades['short'][day]>trades['long'][day]):
                    buy = 1  
                if buy == True:
                    if (cond1_sell & cond2_sell) | (trades['short'][day]<trades['long'][day]):
                        buy = 0
                        hold=True
                        diasParado=day
                    trades['signalBuy'][day]=buy

            if buy==False & hold==False:
                if (cond1_sell & cond2_sell) | (trades['short'][day]<trades['long'][day]):
                    sold = 1      
                if sold == True:
                    if (cond1_buy & cond2_buy) | (trades['short'][day]>trades['long'][day]):
                        sold = 0
                        hold=True
                        diasParado=day
                    trades['signalSell'][day]=sold
            hold=False


#        buyed=0
#        sold=0
#        buy=0    
#            
#        for day in range(1,len(trades)):
#            cond1_buy=trades['Close'].iloc[day-1] > trades['bol_bands_lower'].iloc[day-1]
#            cond2_buy=trades['Close'].iloc[day] < trades['bol_bands_lower'].iloc[day]  
#            cond1_sell=trades['Close'].iloc[day-1] < trades['bol_bands_middle'].iloc[day-1]
#            cond2_sell=trades['Close'].iloc[day] > trades['bol_bands_middle'].iloc[day]
#            
#            if sold==False:
#                if (cond1_buy & cond2_buy) | (trades['short'][day]>trades['long'][day]):
#                    buy = 1
#                if buy == True:
#                    if (cond1_sell & cond2_sell) & (trades['short'][day]<trades['long'][day]):
#                        buy = 0
#                    trades['signalBuy'][day]=buy
#
#            if buy==False:
#                if (cond1_sell & cond2_sell) | (trades['short'][day]<trades['long'][day]):
#                    sold = 1
#                if sold==True:
#                    if (cond1_buy & cond2_buy) & (trades['short'][day]>trades['long'][day]):
#                        sold=0
#                    trades['signalSell'][day]=sold
    #se estiver comprado no último dia vende

    if trades['signalBuy'][len(trades['signalBuy'])-1]==1 or trades['signalSell'][len(trades['signalSell'])-1]==1:
        trades['signalBuy'][len(trades['signalBuy'])-1]=0
        trades['signalSell'][len(trades['signalSell'])-1]=0

    # Posição dos trades
    trades['positionsBuy'] = trades['signalBuy'].diff()
    trades['positionsSell'] = trades['signalSell'].diff()

    fig = plt.subplots(figsize=(30,25))
    plt.style.use('seaborn-whitegrid')

    ax1 = plt.subplot(321)
    ax1.plot( trades.Close, 'y', label = 'Close')
    ax1.plot( trades.bol_bands_middle, 'k:', label = 'middle')
    ax1.plot( trades.bol_bands_upper, 'm', label = 'upper')
    ax1.plot( trades.bol_bands_lower, 'g', label = 'lower')
    ax1.plot(trades['short'][-494:], label='Short')
    ax1.plot(trades['long'][-494:],label='Long')

    ax1.plot(trades.loc[trades.positionsBuy == 1.0].index, 
             trades.bol_bands_lower[trades.positionsBuy == 1.0],
             '^', markersize=10, color='m',label='Buy')
    ax1.plot(trades.loc[trades.positionsBuy == -1.0].index, 
             trades.bol_bands_lower[trades.positionsBuy == -1.0],
             'v', markersize=10, color='k',label='Sell') 

    ax1.plot(trades.loc[trades.positionsSell == 1.0].index, 
             trades.bol_bands_lower[trades.positionsSell == 1.0],
             '^', markersize=10, color='r',label='Sell')
    ax1.plot(trades.loc[trades.positionsSell == -1.0].index, 
             trades.bol_bands_lower[trades.positionsSell == -1.0],
             'v', markersize=10, color='g',label='Buy') 

    ax1.set_ylabel("Preço [R$]",fontsize=25)
    ax1.set_xlabel("Data",fontsize=25)
    ax1.set_title("Médias " + stk,fontsize=25)
    ax1.legend(loc='upper left',fontsize=25)

    #trades['Close_recorte']=trades['Close']*trades['signal']

    #trades['Close_recorte'][trades['Close_recorte']==0]=16

    #ax3 = plt.subplot(312)
    #ax3.plot(trades['Close_recorte'], label='Close')
    #ax3.set_ylabel("Preço [R$]",fontsize=25)
    #ax3.set_xlabel("Data",fontsize=25)
    #ax3.set_title("Médias " + stk,fontsize=25)
    #ax3.legend(loc='upper left',fontsize=25)
    

    ax5 = plt.subplot(323)
    #
    pos_buy_buyed=np.where(trades['positionsBuy']==1)
    pos_sell_buyed=trades['positionsBuy']==-1
    days_buy_buyed=trades['Close'][trades['positionsBuy']==1].index
    days_sell_buyed=trades['Close'][trades['positionsBuy']==-1].index
    distance_buyed=days_sell_buyed-days_buy_buyed
    #
    pos_buy_sold=np.where(trades['positionsSell']==1)
    pos_sell_sold=trades['positionsSell']==-1
    days_buy_sold=trades['Close'][trades['positionsSell']==1].index
    days_sell_sold=trades['Close'][trades['positionsSell']==-1].index
    distance_sold=days_sell_sold-days_buy_sold
    #
    buy_trade=[]
    sell_trade=[]
    # Gráfico para comprado
    for i in range(len(days_buy_buyed)):
        values_2=trades['Close'][trades['Close'].index==(days_buy_buyed[i]+distance_buyed[i])].values[0]
        values_1=trades['Close'][trades['Close'].index==days_buy_buyed[i]].values[0]
        buy_trade.append(100*(values_2-values_1)/values_1)

    valuesbuy = np.array(buy_trade)
    values1buy=buy_trade

    idx = np.array(list(range(len(buy_trade)))) 
    clrs = ['red' if (x < 0) else 'blue' for x in valuesbuy]
    sns.barplot(x=idx, y=valuesbuy, palette=clrs) 

    plt.ylabel('Retorno R[%] Comprado',fontsize=25)
    plt.xlabel('Trades Comprado',fontsize=25)

    ax52 = ax5.twinx()

    ax52.plot(sumper(valuesbuy),'gold',linewidth=4)

    plt.ylabel('Retorno Acumulado R[$]',fontsize=25)
    plt.xlabel('Trades',fontsize=25)
    plt.title('Retorno ' + " " + stk,fontsize=25)
    
    #Gráfico para vendido
    ax5segundo = plt.subplot(325)
    
    for i in range(len(days_buy_sold)):
        values_2=trades['Close'][trades['Close'].index==(days_buy_sold[i]+distance_sold[i])].values[0]
        values_1=trades['Close'][trades['Close'].index==days_buy_sold[i]].values[0]
        sell_trade.append(100*(values_1-values_2)/values_1)

    valuessell = np.array(sell_trade)
    values1sell=sell_trade

    idx = np.array(list(range(len(sell_trade)))) 
    clrs = ['red' if (x < 0) else 'blue' for x in valuessell]
    sns.barplot(x=idx, y=valuessell, palette=clrs) 

    plt.ylabel('Retorno R[%] Vendido',fontsize=25)
    plt.xlabel('Trades Vendido',fontsize=25)

    ax5segundo2 = ax5segundo.twinx()

    ax5segundo2.plot(sumper(valuessell),'gold',linewidth=4)

    plt.ylabel('Retorno Acumulado R[$]',fontsize=25)
    plt.xlabel('Trades',fontsize=25)
    plt.title('Retorno ' + " " + stk,fontsize=25)
    
    plt.tight_layout()
    
    a=1
    for kint in range(len(valuesbuy)):
        a*=((valuesbuy[kint]/100)+1)
        if valuesbuy[kint]>0:
            taxa_acerto+=1
            ftr_pos*=((valuesbuy[kint]/100)+1)
        else:
            ftr_neg*=((valuesbuy[kint]/-100)+1)
    soma_retornosA+=(a*CapitalInicial[i12])
    
    b=1
    for kint in range(len(valuessell)):
        b*=((valuessell[kint]/100)+1)
        if valuessell[kint]>0:
            taxa_acerto+=1
            ftr_pos*=((valuessell[kint]/100)+1)
        else:
            ftr_neg*=((valuessell[kint]/-100)+1)
    soma_retornosB+=(b*CapitalInicial[i12])
    
    
    aux.append(((a*b)-1)*100)
  
    i12+=1
    
    num_trades+=(len(valuesbuy)+len(valuessell))

for iteracao in range(len(aux)):
    print(aux[iteracao], stocks[iteracao])
    
taxa_acerto=(taxa_acerto/num_trades)*100

fator_lucro= ((ftr_pos-1)*100000)/((ftr_neg-1)*100000)

In [ ]:
print(round(fator_lucro,2))

In [ ]:
print("Taxa de acerto: {0}%" .format(round(taxa_acerto,2)))

# Aplicando no período de teste

In [ ]:
out_teste

# Operar comprado e vendido

In [ ]:
Trades=[]
Trades.append(0)
Trades.append(0)
Trades.append(0)
Trades.append(0)
i12=0
soma_retornosA=0
soma_retornosB=0

fator_lucro=1
ftr_pos=1
ftr_neg=1

taxa_acerto=0
num_trades=0

for stk in stocks:    
    short=melhor_short
    long=melhor_long

    trades = pd.DataFrame(index=out_teste.index)
    trades['Close']=out_teste[stk].values

    trades.reset_index(inplace=True)
    bollinger_bands(trades, melhor_trend_periods, melhor_deviation)

    trades['short'] = trades['Close'].rolling(window=short).mean()
    trades['long'] = trades['Close'].rolling(window=long).mean()

    trades['signalBuy']=0
    trades['signalSell']=0

    buy=0
    buyed=0
    sold=0
    hold=False
    diasParado=0
    for day in range(1,len(trades)):
        if day>(diasParado+5):
            cond1_buy=trades['Close'].iloc[day-1] > trades['bol_bands_lower'].iloc[day-1]
            cond2_buy=trades['Close'].iloc[day] < trades['bol_bands_lower'].iloc[day]  
            cond1_sell=trades['Close'].iloc[day-1] < trades['bol_bands_middle'].iloc[day-1]
            cond2_sell=trades['Close'].iloc[day] > trades['bol_bands_middle'].iloc[day]

            if sold==False & hold==False:
                if (cond1_buy & cond2_buy) | (trades['short'][day]>trades['long'][day]):
                    buy = 1  
                if buy == True:
                    if (cond1_sell & cond2_sell) | (trades['short'][day]<trades['long'][day]):
                        buy = 0
                        hold=True
                        diasParado=day
                    trades['signalBuy'][day]=buy

            if buy==False & hold==False:
                if (cond1_sell & cond2_sell) | (trades['short'][day]<trades['long'][day]):
                    sold = 1      
                if sold == True:
                    if (cond1_buy & cond2_buy) | (trades['short'][day]>trades['long'][day]):
                        sold = 0
                        hold=True
                        diasParado=day
                    trades['signalSell'][day]=sold
            hold=False


#        buyed=0
#        sold=0
#        buy=0    
#            
#        for day in range(1,len(trades)):
#            cond1_buy=trades['Close'].iloc[day-1] > trades['bol_bands_lower'].iloc[day-1]
#            cond2_buy=trades['Close'].iloc[day] < trades['bol_bands_lower'].iloc[day]  
#            cond1_sell=trades['Close'].iloc[day-1] < trades['bol_bands_middle'].iloc[day-1]
#            cond2_sell=trades['Close'].iloc[day] > trades['bol_bands_middle'].iloc[day]
#            
#            if sold==False:
#                if (cond1_buy & cond2_buy) | (trades['short'][day]>trades['long'][day]):
#                    buy = 1
#                if buy == True:
#                    if (cond1_sell & cond2_sell) & (trades['short'][day]<trades['long'][day]):
#                        buy = 0
#                    trades['signalBuy'][day]=buy
#
#            if buy==False:
#                if (cond1_sell & cond2_sell) | (trades['short'][day]<trades['long'][day]):
#                    sold = 1
#                if sold==True:
#                    if (cond1_buy & cond2_buy) & (trades['short'][day]>trades['long'][day]):
#                        sold=0
#                    trades['signalSell'][day]=sold
    #se estiver comprado no último dia vende

    if trades['signalBuy'][len(trades['signalBuy'])-1]==1 or trades['signalSell'][len(trades['signalSell'])-1]==1:
        trades['signalBuy'][len(trades['signalBuy'])-1]=0
        trades['signalSell'][len(trades['signalSell'])-1]=0

    # Posição dos trades
    trades['positionsBuy'] = trades['signalBuy'].diff()
    trades['positionsSell'] = trades['signalSell'].diff()

    fig = plt.subplots(figsize=(30,25))
    plt.style.use('seaborn-whitegrid')

    ax1 = plt.subplot(321)
    ax1.plot( trades.Close, 'y', label = 'Close')
    ax1.plot( trades.bol_bands_middle, 'k:', label = 'middle')
    ax1.plot( trades.bol_bands_upper, 'm', label = 'upper')
    ax1.plot( trades.bol_bands_lower, 'g', label = 'lower')
    ax1.plot(trades['short'][-494:], label='Short')
    ax1.plot(trades['long'][-494:],label='Long')

    ax1.plot(trades.loc[trades.positionsBuy == 1.0].index, 
             trades.bol_bands_lower[trades.positionsBuy == 1.0],
             '^', markersize=10, color='m',label='Buy')
    ax1.plot(trades.loc[trades.positionsBuy == -1.0].index, 
             trades.bol_bands_lower[trades.positionsBuy == -1.0],
             'v', markersize=10, color='k',label='Sell') 

    ax1.plot(trades.loc[trades.positionsSell == 1.0].index, 
             trades.bol_bands_lower[trades.positionsSell == 1.0],
             '^', markersize=10, color='r',label='Sell')
    ax1.plot(trades.loc[trades.positionsSell == -1.0].index, 
             trades.bol_bands_lower[trades.positionsSell == -1.0],
             'v', markersize=10, color='g',label='Buy') 

    ax1.set_ylabel("Preço [R$]",fontsize=25)
    ax1.set_xlabel("Data",fontsize=25)
    ax1.set_title("Médias " + stk,fontsize=25)
    ax1.legend(loc='upper left',fontsize=25)

    #trades['Close_recorte']=trades['Close']*trades['signal']

    #trades['Close_recorte'][trades['Close_recorte']==0]=16

    #ax3 = plt.subplot(312)
    #ax3.plot(trades['Close_recorte'], label='Close')
    #ax3.set_ylabel("Preço [R$]",fontsize=25)
    #ax3.set_xlabel("Data",fontsize=25)
    #ax3.set_title("Médias " + stk,fontsize=25)
    #ax3.legend(loc='upper left',fontsize=25)
    

    ax5 = plt.subplot(323)
    #
    pos_buy_buyed=np.where(trades['positionsBuy']==1)
    pos_sell_buyed=trades['positionsBuy']==-1
    days_buy_buyed=trades['Close'][trades['positionsBuy']==1].index
    days_sell_buyed=trades['Close'][trades['positionsBuy']==-1].index
    distance_buyed=days_sell_buyed-days_buy_buyed
    #
    pos_buy_sold=np.where(trades['positionsSell']==1)
    pos_sell_sold=trades['positionsSell']==-1
    days_buy_sold=trades['Close'][trades['positionsSell']==1].index
    days_sell_sold=trades['Close'][trades['positionsSell']==-1].index
    distance_sold=days_sell_sold-days_buy_sold
    #
    buy_trade=[]
    sell_trade=[]
    # Gráfico para comprado
    for i in range(len(days_buy_buyed)):
        values_2=trades['Close'][trades['Close'].index==(days_buy_buyed[i]+distance_buyed[i])].values[0]
        values_1=trades['Close'][trades['Close'].index==days_buy_buyed[i]].values[0]
        buy_trade.append(100*(values_2-values_1)/values_1)

    valuesbuy = np.array(buy_trade)
    values1buy=buy_trade

    idx = np.array(list(range(len(buy_trade)))) 
    clrs = ['red' if (x < 0) else 'blue' for x in valuesbuy]
    sns.barplot(x=idx, y=valuesbuy, palette=clrs) 

    plt.ylabel('Retorno R[%] Comprado',fontsize=25)
    plt.xlabel('Trades Comprado',fontsize=25)

    ax52 = ax5.twinx()

    ax52.plot(sumper(valuesbuy),'gold',linewidth=4)

    plt.ylabel('Retorno Acumulado R[$]',fontsize=25)
    plt.xlabel('Trades',fontsize=25)
    plt.title('Retorno ' + " " + stk,fontsize=25)
    
    #Gráfico para vendido
    ax5segundo = plt.subplot(325)
    
    for i in range(len(days_buy_sold)):
        values_2=trades['Close'][trades['Close'].index==(days_buy_sold[i]+distance_sold[i])].values[0]
        values_1=trades['Close'][trades['Close'].index==days_buy_sold[i]].values[0]
        sell_trade.append(100*(values_1-values_2)/values_1)

    valuessell = np.array(sell_trade)
    values1sell=sell_trade

    idx = np.array(list(range(len(sell_trade)))) 
    clrs = ['red' if (x < 0) else 'blue' for x in valuessell]
    sns.barplot(x=idx, y=valuessell, palette=clrs) 

    plt.ylabel('Retorno R[%] Vendido',fontsize=25)
    plt.xlabel('Trades Vendido',fontsize=25)

    ax5segundo2 = ax5segundo.twinx()

    ax5segundo2.plot(sumper(valuessell),'gold',linewidth=4)

    plt.ylabel('Retorno Acumulado R[$]',fontsize=25)
    plt.xlabel('Trades',fontsize=25)
    plt.title('Retorno ' + " " + stk,fontsize=25)
    
    plt.tight_layout()
    
    a=1
    for kint in range(len(valuesbuy)):
        a*=((valuesbuy[kint]/100)+1)
        if valuesbuy[kint]>0:
            taxa_acerto+=1
            ftr_pos*=((valuesbuy[kint]/100)+1)
        else:
            ftr_neg*=((valuesbuy[kint]/-100)+1)
    soma_retornosA+=(a*CapitalInicial[i12])
    
    b=1
    for kint in range(len(valuessell)):
        b*=((valuessell[kint]/100)+1)
        if valuessell[kint]>0:
            taxa_acerto+=1
            ftr_pos*=((valuessell[kint]/100)+1)
        else:
            ftr_neg*=((valuessell[kint]/-100)+1)
    soma_retornosB+=(b*CapitalInicial[i12])
    
    
    aux=((a*b))
    
    patrimonio=[]
    patrimonio.append(CapitalInicial[i12]*100000)
    patrimonio.append(patrimonio[0]*aux)
    Trades[i12]=patrimonio
    
    num_trades+=(len(valuesbuy)+len(valuessell))
    
    i12+=1
    
taxa_acerto=(taxa_acerto/num_trades)*100

fator_lucro= ((ftr_pos-1)*100000)/((ftr_neg-1)*100000)

In [ ]:
num_trades

In [ ]:
print(round(fator_lucro,2))

In [ ]:
print("Taxa de acerto: {0}%" .format(round(taxa_acerto,2)))

In [ ]:
numTrades=[a for a in range(len(Trades[0]))]
irbr3=pd.DataFrame(index=numTrades)
irbr3['Patrimonio IRBR3']=Trades[0]
#irbr3.loc[-1]=[(CapitalInicial[0]*100000)]
#irbr3.index=irbr3.index+1
#irbr3=irbr3.sort_index()


numTrades=[a for a in range(len(Trades[1]))]
rent3=pd.DataFrame(index=numTrades)
rent3['Patrimonio RENT3']=Trades[1]
#rent3.loc[-1]=[(CapitalInicial[1]*100000)]
#rent3.index=rent3.index+1
#rent3=rent3.sort_index()

numTrades=[a for a in range(len(Trades[2]))]
mglu3=pd.DataFrame(index=numTrades)
mglu3['Patrimonio MGLU3']=Trades[2]
#mglu3.loc[-1]=[(CapitalInicial[2]*100000)]
#mglu3.index=mglu3.index+1
#mglu3=mglu3.sort_index()

numTrades=[a for a in range(len(Trades[3]))]
jbss3=pd.DataFrame(index=numTrades)
jbss3['Patrimonio JBSS3']=Trades[3]
#jbss3.loc[-1]=[(CapitalInicial[3]*100000)]
#jbss3.index=jbss3.index+1
#jbss3=jbss3.sort_index()

In [ ]:
#ax1 = plt.subplot(321)
irbr3.plot()
#ax2 = plt.subplot(322)
rent3.plot()
#ax3 = plt.subplot(323)
mglu3.plot()
#ax4 = plt.subplot(324)
jbss3.plot()

In [ ]:
patrimonio=0
for a in range(len(Trades)):
    patrimonio+=Trades[a][-1]
print("Patrimônio final: R$" + str(round(patrimonio,2)))

# Questão 4

## Estratégia 1
### fator lucro = 3.25 
### Taxa de acerto: 36.67%
### Patrimônio final: R$137.928,14
### Número de trades: 30

## Estratégia 2
### fator lucro = 3.46
### Taxa de acerto: 46.15%
### Patrimônio final: R$111.979,58
### Número de trades: 3

## Estratégia 3
### fator lucro = 3.98
### Taxa de acerto: 50.0%
### Patrimônio final: R$152.633,08
### Número de trades: 48

## Condição Buy&Hold
### Patrimônio final: R$80.748,77

# Se ficasse buy-hold

In [ ]:
out_teste

In [ ]:
buy_hold=0
aux=0
for iteracao in range(len(stocks)):
    aux=(CapitalInicial[iteracao]*100000)
    aux/=out_teste[stocks[iteracao]][0]
    aux*=out_teste[stocks[iteracao]][-1]
    buy_hold+=aux
print("Patrimônio final: R$" + str(round(buy_hold,2)))